# Knapsack Optimization with KOPPU

This notebook demonstrates solving the Knapsack problem, a constrained optimization problem. We aim to maximize the value of items in a knapsack without exceeding its capacity.

In [ ]:
import pykoppu as pk

# 1. Define Items and Capacity
items = [
    {'name': 'Item 1', 'weight': 2, 'value': 10},
    {'name': 'Item 2', 'weight': 3, 'value': 15},
    {'name': 'Item 3', 'weight': 5, 'value': 20},
    {'name': 'Item 4', 'weight': 7, 'value': 25},
    {'name': 'Item 5', 'weight': 1, 'value': 5},
    {'name': 'Item 6', 'weight': 4, 'value': 12},
    {'name': 'Item 7', 'weight': 1, 'value': 3}
]
capacity = 10
penalty = 10.0 # Penalty strength for constraint violation

# 2. Create Knapsack Problem
print("Mapping to OPU...")
problem = pk.problems.logistics.Knapsack(items, capacity, penalty)

# 3. Solve
print("Solving...")
# We set the simulation duration to 500ms using the 't' parameter
process = pk.oos.Process(problem, t=500)
result = process.run()

# 4. Display OPU Dynamics
print("\nVisualizing OPU Dynamics...")
result.plot()

# 5. Analyse and Display Solution
print("\nVisualizing Solution...")
# We use a threshold of 0.3 to determine if an item is selected
problem.plot(result, threshold=0.3)